<a href="https://colab.research.google.com/github/gita-liferishi/dp-llm-unlearning/blob/main/Yelp_Dataset_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

In [2]:
import pandas as pd

splits = {'train': 'yelp_review_full/train-00000-of-00001.parquet', 'test': 'yelp_review_full/test-00000-of-00001.parquet'}
df = pd.read_parquet("hf://datasets/Yelp/yelp_review_full/" + splits["train"])
df.head()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


,label,text
0,4,dr. goldberg offers everything i look for in a...
1,1,"Unfortunately, the frustration of being Dr. Go..."
2,3,Been going to Dr. Goldberg for over 10 years. ...
3,3,Got a letter in the mail last week that said D...
4,0,I don't know what Dr. Goldberg was like before...


In [3]:
df.isnull().sum()

,0
label,0
text,0


In [4]:
df.dropna(inplace=True)

In [5]:
df = df.iloc[:100001,:]

In [6]:
df.shape

(100001, 2)

# Data Prep

In [ ]:
df_mod = df.copy()

In [ ]:
# converting text to lower case
df_mod = df_mod.applymap(lambda x: x.lower() if isinstance(x, str) else x)

<ipython-input-14-20369b8d26b7>:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_mod = df_mod.applymap(lambda x: x.lower() if isinstance(x, str) else x)


In [ ]:
# removing contractions
contractions = {
    "ain't": "is not", "aren't": "are not", "can't": "cannot", "couldn't": "could not", "didn't": "did not",
    "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",
    "he'd": "he would", "he'll": "he will", "he's": "he is", "how'd": "how did", "how'll": "how will",
    "how's": "how is", "I’d": "I would", "I'll": "I will", "I'm": "I am", "I've": "I have", "I’m": "I am",
    "isn't": "is not", "it'd": "it would", "it'll": "it will", "it's": "it is", "let's": "let us", "ma'am": "madam",
    "mightn't": "might not", "mustn't": "must not", "needn't": "need not", "shan't": "shall not", "she'd": "she would",
    "she'll": "she will", "she's": "she is", "shouldn't": "should not", "that'd": "that would", "that'll": "that will",
    "that's": "that is", "there'd": "there would", "there'll": "there will", "there's": "there is", "they'd": "they would",
    "they'll": "they will", "they're": "they are", "they've": "they have", "wasn't": "was not", "weren't": "were not",
    "what'll": "what will", "what's": "what is", "what've": "what have", "where'd": "where did", "where'll": "where will",
    "where's": "where is", "who'd": "who would", "who'll": "who will", "who's": "who is", "who've": "who have",
    "why'd": "why did", "why'll": "why will", "why's": "why is", "won't": "will not", "wouldn't": "would not",
}
def expand_contractions(text):
    # Check if the input is a float
    if isinstance(text, float):
        return text

    for contraction, expanded in contractions.items():
        text = text.replace(contraction, expanded)

    return text

# apply
df_mod['text'] = df_mod['text'].apply(expand_contractions)

In [ ]:
# removing puntuations
df_mod = df_mod.replace(r'[^\w\s]', '', regex=True)

In [ ]:
# stop words removal
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
stop = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# for all col
df_mod['text'] = df_mod['text'].apply(lambda x: ' '.join([word for word in str(x).split() if word not in stop]) if x is not None else '')


In [ ]:
# stemming
import nltk
from nltk.stem import PorterStemmer
nltk.download('punkt_tab')
stemmer = PorterStemmer()

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [ ]:
def stem_text(text):
    if isinstance(text, str):
        words = nltk.word_tokenize(text)
        stemmed_words = [stemmer.stem(word) for word in words]
        return ' '.join(stemmed_words)


df_mod['text'] = df_mod['text'].apply(stem_text)

In [7]:
df_mod.head(5)

In [8]:
df_mod.shape

# BERT

In [ ]:
pip install transformers datasets torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 104.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 81.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 60.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from transformers import BertTokenizer
from torch.utils.data import Dataset, DataLoader
import torch
from transformers import BertForSequenceClassification, AdamW
from torch.nn import CrossEntropyLoss

In [ ]:
# Encoding the target variable 'Popularity'
le = LabelEncoder()
df['label'] = le.fit_transform(df['label'])


In [ ]:
from sklearn.model_selection import train_test_split

# Split data (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(
    df['text'], df['label'], test_size=0.2, random_state=42
)

In [ ]:
from transformers import BertTokenizer

# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Define a function to tokenize the text data
def tokenize_texts(texts, max_length=512):
    return tokenizer.batch_encode_plus(
        texts,
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_tensors='pt'  # Return PyTorch tensors
    )

# Tokenize the training and test data
train_encodings = tokenize_texts(X_train.tolist())
test_encodings = tokenize_texts(X_test.tolist())

torch.save(train_encodings, 'train_encodings.pt')
torch.save(test_encodings, 'test_encodings.pt')

In [ ]:
train_encodings = torch.load('train_encodings.pt', weights_only=False)
test_encodings = torch.load('test_encodings.pt', weights_only=False)

In [ ]:
y_train_tensor = torch.tensor(y_train.tolist(), dtype=torch.long)
y_test_tensor = torch.tensor(y_test.tolist(), dtype=torch.long)

In [ ]:
class TextDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            "input_ids": self.encodings["input_ids"][idx],
            "attention_mask": self.encodings["attention_mask"][idx],
            "labels": self.labels[idx]
        }

In [ ]:
# Create Dataset objects
train_dataset = TextDataset(train_encodings, y_train_tensor)
test_dataset = TextDataset(test_encodings, y_test_tensor)

# DataLoader
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=16, shuffle=False)

In [ ]:
import torch
import torch.nn as nn
from transformers import BertModel

class BERTForClassification(nn.Module):
    """
    BERT-based model for multi-class text classification.
    """

    def __init__(self, bert_model_name='bert-base-uncased', num_classes=5):
        super(BERTForClassification, self).__init__()

        # Initialize BERT model
        self.bert = BertModel.from_pretrained(bert_model_name)

        # Define additional layers for classification
        self.dropout = nn.Dropout(p=0.3)  # Dropout layer to prevent overfitting
        self.fc1 = nn.Linear(self.bert.config.hidden_size, 128)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(128, 64)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(64, num_classes)

    def forward(self, input_ids, attention_mask):
        # Get the pooled output from BERT
        bert_output = self.bert(input_ids=input_ids, attention_mask=attention_mask).pooler_output

        # Classification head
        x = self.dropout(bert_output)
        x = self.relu1(self.fc1(x))
        x = self.dropout(x)
        x = self.relu2(self.fc2(x))
        x = self.dropout(x)
        logits = self.fc3(x)

        return logits

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import BertTokenizer, BertModel
from sklearn.metrics import f1_score, confusion_matrix
import numpy as np
from tqdm import tqdm  # For progress display

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Instantiate the model
model = BERTForClassification().to(device)

# Optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=1e-5)
criterion = nn.CrossEntropyLoss()

# Optional: Learning rate scheduler
# scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)

# Training loop
num_epochs = 1  # Adjust as needed

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    print(f"Epoch {epoch + 1}/{num_epochs}")
    for batch in tqdm(train_loader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()  # Reset gradients

        # Forward pass
        outputs = model(input_ids, attention_mask)
        loss = criterion(outputs, labels)

        # Backward pass
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    # Optional: Step the scheduler
    # scheduler.step()

    avg_train_loss = total_loss / len(train_loader)
    print(f"Training Loss: {avg_train_loss:.4f}")

    # Add validation step here if you have a validation set:
    # validate(model, val_loader, criterion, device)

# Evaluation loop
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in tqdm(test_loader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask)
        _, predicted = torch.max(outputs, dim=1)

        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Convert to NumPy arrays
all_preds = np.array(all_preds)
all_labels = np.array(all_labels)

# Calculate F1-score
f1 = f1_score(all_labels, all_preds, average='weighted')
print(f"F1 Score: {f1:.4f}")

# Confusion Matrix
conf_matrix = confusion_matrix(all_labels, all_preds)
print("Confusion Matrix:\n", conf_matrix)

# Save model, optionally only if it has improved the best validation score
torch.save(model.state_dict(), 'bert_classification_model.pth')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Epoch 1/1


100%|██████████| 5000/5000 [2:05:52<00:00,  1.51s/it]


Training Loss: 1.1797


100%|██████████| 1251/1251 [10:42<00:00,  1.95it/s]


F1 Score: 0.5789
Confusion Matrix:
 [[3402 1085   72   13   72]
 [ 774 2417  619   74   72]
 [ 127 1113 2077  466  184]
 [  51  250 1262 1389  973]
 [  75   92  297  697 2348]]
